In [ ]:
import requests
import json
import os
from requests_toolbelt import MultipartEncoder, MultipartEncoderMonitor
from tqdm import tqdm

# --- 配置区 ---
url = "http://10.129.9.44:8002/api/upload"
model_file_path = "malware_classifier.pth"

# 检查文件是否存在
if not os.path.exists(model_file_path):
    print(f"❌ 找不到文件: {model_file_path}")
    exit()

# 获取文件大小用于进度条显示
file_size = os.path.getsize(model_file_path)

# --- 进度条回调函数 ---
def create_callback(encoder_tqdm):
    def callback(monitor):
        # monitor.bytes_read 是已经读取的字节数
        # update 需要传入增量值
        encoder_tqdm.update(monitor.bytes_read - encoder_tqdm.n)
    return callback

# --- 上传逻辑 ---
print(f"🚀 开始上传 {model_file_path} ({file_size / (1024*1024):.2f} MB)...")

with open(model_file_path, "rb") as f:
    # 1. 构造多部分表单编码器
    encoder = MultipartEncoder(
        fields={'model': (os.path.basename(model_file_path), f, 'application/octet-stream')}
    )
    
    # 2. 构造进度条（tqdm）
    with tqdm(total=encoder.len, unit='B', unit_scale=True, unit_divisor=1024, desc="Uploading") as bar:
        
        # 3. 构造监视器，并绑定回调函数
        monitor = MultipartEncoderMonitor(encoder, create_callback(bar))
        
        # 4. 发送请求
        # 注意：这里的 data 必须传入 monitor，并且必须手动指定 Content-Type
        try:
            response = requests.post(
                url, 
                data=monitor, 
                headers={'Content-Type': monitor.content_type}
            )
            
            # 打印结果
            print("\n✅ 上传完成！服务器响应：")
            print(json.dumps(response.json(), indent=4))
            
        except Exception as e:
            print(f"\n❌ 上传失败: {e}")